# Обработка датасета

## Библиотеки и скрипты

In [1]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Библиотека для предобработки датасетов
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_preprocessing import DataPreprocessor
dp = DataPreprocessor()

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

## Обзор данных

In [2]:
# Загружаем датасет
df = pd.read_csv('../data/wp_cn_requests_price.csv')
# Удаляем дубликаты
df = df.drop_duplicates(subset='id_session', keep="first")
df = df.drop_duplicates(subset='id_YM', keep="first")

In [3]:
df.head(5)

,id,id_session,id_YM,id_GA,created,location,branch,direction,amount,price
0,15,e70acf53510a6d7e46f75e16bf37a4eb,,NaN,2020-12-23 10:35:19,Yeysk,Выездное обслуживание оборудования,Спецтехника,100,800
68,88,af0c2ce6101731fd9b44046f99677001,160734218558947414,GA1.2.898178360.1607513465,2020-12-24 13:34:18,Moscow,Мобильные сотрудники,Монтажники,50,NaN
71,91,8f2b41da2eaa4728de2a58961fa4b28f,1608812392678591992,GA1.2.710466435.1608812392,2020-12-24 15:20:58,Moscow,Выездное обслуживание оборудования,Обслуживание розничных сетей,10,9500
72,92,0b23e19cc92a1be6413794061b835692,1608813210562849312,GA1.2.1067518638.1608813213,2020-12-24 15:37:04,Voronezh,"Управление самозанятыми, ИП и подрядчиками",Управление ИП и самозанятыми,1,NaN
74,94,ff192463715f0279e7da68abcf7188ef,1608818770263758756,GA1.2.1966351661.1608818770,2020-12-24 17:10:29,Moscow,Выездное обслуживание оборудования,HoReCa,100,80200


In [4]:
df.shape

(1004, 10)

## Обработка данных

In [5]:
# Удаление ненужных данных
columns = ['id', 'id_session']
df = dp.drop_col(df, columns)

# Убираем 2020 год (там одни тесты)
df = df[df['created'].str.contains('2021.*')]

In [6]:
df.columns

Index(['id_YM', 'id_GA', 'created', 'location', 'branch', 'direction',
       'amount', 'price'],
      dtype='object')

In [7]:
# Переименование столбцов
df.columns = dp.get_translite(df.columns, 'st_')
df.columns

Index(['st_id_ym', 'st_id_ga', 'st_created', 'st_location', 'st_branch',
       'st_direction', 'st_amount', 'st_price'],
      dtype='object')

In [8]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 943 entries, 226 to 2812
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   st_id_ym      943 non-null    object
 1   st_id_ga      915 non-null    object
 2   st_created    943 non-null    object
 3   st_location   943 non-null    object
 4   st_branch     943 non-null    object
 5   st_direction  943 non-null    object
 6   st_amount     943 non-null    int64 
 7   st_price      943 non-null    object
dtypes: int64(1), object(7)
memory usage: 66.3+ KB


st_id_ym         0
st_id_ga        28
st_created       0
st_location      0
st_branch        0
st_direction     0
st_amount        0
st_price         0
dtype: int64

In [9]:
# Обработка выбросов
df.loc[df['st_amount'] > 2000, 'st_amount'] = df['st_amount'].median()

In [10]:
# Корректировка значений
df.loc[df['st_id_ym'] == '', 'st_id_ym'] = 'undefined'
df.loc[df['st_id_ga'] == '', 'st_id_ga'] = 'undefined'
df.loc[df['st_id_ym'].isnull(), 'st_id_ym'] = 'undefined'
df.loc[df['st_id_ga'].isnull(), 'st_id_ga'] = 'undefined'
df.loc[df['st_price'] == 'По запросу', 'st_price'] = 0
df.loc[df['st_price'] == 0.0, 'st_price'] = df['st_price'].median()

# Поиск и замена по регулярке
df.loc[df['st_location'].str.contains("’"), 'st_location'] = df['st_location'].str.replace("’", "")
df.loc[df['st_location'].str.contains(r"\\'"), 'st_location'] = df['st_location'].str.replace(r"\\'", "")

# Заменим название направления типа "Другое" на название корневой отрасли
df.loc[df['st_direction'] == 'Другое', 'st_direction'] = df['st_branch']

In [11]:
# df.loc[df['st_location'].str.contains("\'")]

In [12]:
# Приведение типов
df = dp.astype_col(df, ['st_amount'], coltype='uint8')
df = dp.astype_col(df, ['st_price'], coltype='float64')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 943 entries, 226 to 2812
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   st_id_ym      943 non-null    object 
 1   st_id_ga      943 non-null    object 
 2   st_created    943 non-null    object 
 3   st_location   943 non-null    object 
 4   st_branch     943 non-null    object 
 5   st_direction  943 non-null    object 
 6   st_amount     943 non-null    uint8  
 7   st_price      943 non-null    float64
dtypes: float64(1), object(6), uint8(1)
memory usage: 92.1+ KB


In [14]:
df.describe()

,st_amount,st_price
count,943.000000,943.000000
mean,17.652174,14229.703075
std,35.881586,31257.275554
min,0.000000,0.000000
25%,3.000000,5000.000000
50%,5.000000,5000.000000
75%,10.000000,10000.000000
max,244.000000,360000.000000


In [16]:
df.head(5)

,st_id_ym,st_id_ga,st_created,st_location,st_branch,st_direction,st_amount,st_price
226,1609586985682049430,GA1.2.934928888.1609586985,2021-01-02 14:30:42,Kazan,Мобильные сотрудники,Строительные бригады,3,5000.0
227,1609610415617206319,GA1.2.233829537.1609610416,2021-01-02 21:01:43,Barnaul,Телекоммуникации и ИТ,Выездной ИТ-сервис,5,5000.0
230,1609623797665217591,GA1.2.52594619.1609623797,2021-01-03 00:46:09,Minsk,"Управление самозанятыми, ИП и подрядчиками",Управление ИП и самозанятыми,3,5000.0
233,160964815239544789,GA1.2.1046709437.1609648153,2021-01-03 07:30:46,Barnaul,Выездное обслуживание оборудования,Кондиционирование и вентиляция,3,5000.0
236,1609662851665134598,GA1.2.300748215.1609662851,2021-01-03 11:35:45,Shakhty,Выездное обслуживание оборудования,Технологическое оборудование,1,5000.0


In [17]:
df.shape

(943, 8)

## Сохранение данных

In [18]:
# Сохранение в файл
df.to_csv('../data/td_site.csv', index=False, encoding='utf-8', sep=';')